In [29]:
# import necessary packages. 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from geopy.geocoders import Nominatim

AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)

In [3]:
# fetch the data from csv file and store it in a variable. 
space_missions_df = pd.read_csv('./Space_Corrected.csv', index_col=0)

In [4]:
# Rename the columns named `Unnamed: 0` to `index` 
space_missions_df = space_missions_df.rename(columns={"Unnamed: 0" : "index", "Company Name": "company_name", "Status Rocket":"status_rocket"," Rocket": "Rocket", "Status Mission": "status_mission"})

In [5]:
space_missions_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4324 entries, 0 to 4323
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           4324 non-null   int64 
 1   company_name    4324 non-null   object
 2   Location        4324 non-null   object
 3   Datum           4324 non-null   object
 4   Detail          4324 non-null   object
 5   status_rocket   4324 non-null   object
 6   Rocket          964 non-null    object
 7   status_mission  4324 non-null   object
dtypes: int64(1), object(7)
memory usage: 304.0+ KB


In [6]:
space_missions_df.describe()

,index
count,4324.000000
mean,2161.500000
std,1248.375611
min,0.000000
25%,1080.750000
50%,2161.500000
75%,3242.250000
max,4323.000000


In [7]:
# find if there's missing values. 
space_missions_df[space_missions_df.isna().any(axis=1)]

,index,company_name,Location,Datum,Detail,status_rocket,Rocket,status_mission
2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
7,7,CASC,"LC-101, Wenchang Satellite Launch Center, China","Thu Jul 23, 2020 04:41 UTC",Long March 5 | Tianwen-1,StatusActive,NaN,Success
13,13,IAI,"Pad 1, Palmachim Airbase, Israel","Mon Jul 06, 2020 01:00 UTC",Shavit-2 | Ofek-16,StatusActive,NaN,Success
28,28,VKS RF,"Site 43/4, Plesetsk Cosmodrome, Russia","Fri May 22, 2020 07:31 UTC",Soyuz 2.1b/Fregat-M | Cosmos 2546,StatusActive,NaN,Success
31,31,ExPace,"Site 95, Jiuquan Satellite Launch Center, China","Tue May 12, 2020 01:16 UTC",Kuaizhou 1A | Xingyun-2 01 (Wuhan) & 02,StatusActive,NaN,Success
...,...,...,...,...,...,...,...,...
4319,4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Wed Feb 05, 1958 07:33 UTC",Vanguard | Vanguard TV3BU,StatusRetired,NaN,Failure
4320,4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA","Sat Feb 01, 1958 03:48 UTC",Juno I | Explorer 1,StatusRetired,NaN,Success
4321,4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Fri Dec 06, 1957 16:44 UTC",Vanguard | Vanguard TV3,StatusRetired,NaN,Failure
4322,4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan","Sun Nov 03, 1957 02:30 UTC",Sputnik 8K71PS | Sputnik-2,StatusRetired,NaN,Success


In [103]:
space_missions_df.head()

,index,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


In [220]:
# check if there any null or nan columns.
space_missions_df.isna().any()

index             False
company_name      False
Location          False
Datum             False
Detail            False
status_rocket     False
Rocket            False
status_mission    False
dtype: bool

In [219]:
# fill nan with default 0.0 in the column Rocket
space_missions_df['Rocket'].fillna(0.0, inplace=True)

In [157]:
# check for duplicated rows from Data frame.
space_missions_df.duplicated().sum()

0

In [110]:
# create two new columns seperating Datum to Date and time.
#create a new column time_zone and store those values in it.
space_missions_df['time_zone'] = space_missions_df['Datum'].str.split(' ').str[5]

#creates a new column taking the original dates and removing timezones from it
space_missions_df['date_um'] = space_missions_df['Datum'].str.replace(space_missions_df['time_zone'][0], '')

#converts the dates into proper datetime format and data type.
space_missions_df['date_um'] = pd.to_datetime(space_missions_df['date_um'], format='mixed')

#create new columns date and time and stores the values in it.
space_missions_df['date']  = space_missions_df['date_um'].dt.date
space_missions_df['time']  = space_missions_df['date_um'].dt.time


In [221]:
space_missions_df.head()

,index,company_name,Location,Datum,Detail,status_rocket,Rocket,status_mission
0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,0.0,Success
3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


#### Unique companies and the company distributions. 

How many unique companies are represented in the dataset? What's the distribution of companies?

In [109]:
space_missions_df['Company Name'].unique()

array(['SpaceX', 'CASC', 'Roscosmos', 'ULA', 'JAXA', 'Northrop', 'ExPace',
       'IAI', 'Rocket Lab', 'Virgin Orbit', 'VKS RF', 'MHI', 'IRGC',
       'Arianespace', 'ISA', 'Blue Origin', 'ISRO', 'Exos', 'ILS',
       'i-Space', 'OneSpace', 'Landspace', 'Eurockot', 'Land Launch',
       'CASIC', 'KCST', 'Sandia', 'Kosmotras', 'Khrunichev', 'Sea Launch',
       'KARI', 'ESA', 'NASA', 'Boeing', 'ISAS', 'SRC', 'MITT', 'Lockheed',
       'AEB', 'Starsem', 'RVSN USSR', 'EER', 'General Dynamics',
       'Martin Marietta', 'Yuzhmash', 'Douglas', 'ASI', 'US Air Force',
       'CNES', 'CECLES', 'RAE', 'UT', 'OKB-586', 'AMBA',
       "Arm??e de l'Air", 'US Navy'], dtype=object)

In [171]:
# all_companies = pd.Series(space_missions_df['Company Name']).value_counts()
space_missions_df['company_name'].value_counts()

company_name
RVSN USSR           1777
Arianespace          279
CASC                 251
General Dynamics     251
NASA                 203
VKS RF               201
US Air Force         161
ULA                  140
Boeing               136
Martin Marietta      114
SpaceX               100
MHI                   84
Northrop              83
Lockheed              79
ISRO                  76
Roscosmos             55
ILS                   46
Sea Launch            36
ISAS                  30
Kosmotras             22
US Navy               17
ISA                   13
Rocket Lab            13
Eurockot              13
ESA                   13
Blue Origin           12
IAI                   11
ExPace                10
ASI                    9
CNES                   8
AMBA                   8
MITT                   7
JAXA                   7
Land Launch            7
UT                     5
KCST                   5
CASIC                  5
Exos                   4
CECLES                 4
Arm??e de l'

In [8]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(space_missions_df.groupby(['company_name', 'Location','Detail']).company_name.agg(['count']))

                                                                                                                        count
company_name     Location                                           Detail                                                   
AEB              VLS Pad, Alc?›ntara Launch Center, Maranh?œo, B... VLS-1 | SACI-2                                          1
                                                                    VLS-1 | SATEC, UNOSAT                                   1
                                                                    VLS-1 | SCD-2A                                          1
AMBA             LA-5B, RAAF Woomera Range Complex, Australia       Redstone Sparta | WRESAT                                1
                 LC-26A, Cape Canaveral AFS, Florida, USA           Juno I | Explorer 1                                     1
                                                                    Juno I | Explorer 2                               

In [13]:
# How many unique launch Locations are present in the dataset. 
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(space_missions_df['Location'].value_counts())

Location
Site 31/6, Baikonur Cosmodrome, Kazakhstan                                                 235
Site 132/1, Plesetsk Cosmodrome, Russia                                                    216
Site 43/4, Plesetsk Cosmodrome, Russia                                                     202
Site 41/1, Plesetsk Cosmodrome, Russia                                                     198
Site 1/5, Baikonur Cosmodrome, Kazakhstan                                                  193
Site 132/2, Plesetsk Cosmodrome, Russia                                                    174
Site 133/3, Plesetsk Cosmodrome, Russia                                                    158
Site 43/3, Plesetsk Cosmodrome, Russia                                                     138
LC-39A, Kennedy Space Center, Florida, USA                                                 120
ELA-2, Guiana Space Centre, French Guiana, France                                          118
SLC-40, Cape Canaveral AFS, Florida, USA 